In [14]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep
from bs4 import BeautifulSoup
import requests
import re

In [15]:
# Inicializar un navegador web Chrome a través de WebDriver
driver = webdriver.Chrome()

# Navegar a la página web de AEMET que contiene información del tiempo para municipios
driver.get("https://deportesmoya.es/?gclid=CjwKCAjw1t2pBhAFEiwA_-A-NOgfedASQBU_NF_uwwA7F24sXqVGuoG3VTtZ2z9DOJuugxH6F4WrhRoCU3YQAvD_BwE")

# Maximizar la ventana del navegador
driver.maximize_window()

# Pausa de 3 segundos
sleep(3)

# Aceptar las cookies en la página web
driver.find_element("css selector", "#soycontrolcookies > div.soycontrolcookies_header > div.soycontrolcookies_buttons > button.btn.btn-default.accept-button").click()

# Realizar un scroll hacia abajo en la página web utilizando execute_script
# En este caso, el scroll desplazará la página hasta que el eje vertical tenga un valor de 1000 píxeles
driver.execute_script("window.scrollTo(0,1000)")

# Pausa de 3 segundos
sleep(3)

# preguntar al usuario que quiere buscar en la página de deportes Moya. 
eleccion_usuario = input("Que producto quieres encontrar en la página de deportes Moya")

# Ingresar la elección del usuario en el campo de búsqueda y presionar Enter
driver.find_element("css selector", "#search_widget > form > input.ui-autocomplete-input").send_keys(eleccion_usuario, Keys.ENTER)

# Pausa de 3 segundos
sleep(3)

# Crear una lista vacía para almacenar las URLs de productos
lista_url = []

# Iterar desde 1 hasta 10 (inclusive)
for i in range(1, 11 ):
    # En cada iteración, obtener la URL del producto y agregarla a la lista
    lista_url.append(driver.find_element("xpath", f'//*[@id="js-product-list"]/div/article[{i}]/div/div[1]/h3/a').get_attribute("href"))

# cerrar el navegador
driver.close()

In [16]:
# Crear un diccionario para almacenar información de zapatillas.
diccionario_zapatillas = {
    "zapatilla": [],
    "precio_rebajado": [],
    "precio_original": []
}

# Iterar a través de las URL en la lista_url.
for url in lista_url:
    # Hacer una solicitud GET a la URL especificada.
    res_tienda = requests.get(url)

    # Crear un objeto BeautifulSoup para analizar el contenido HTML de la página.
    sopa_tienda = BeautifulSoup(res_tienda.content, "html.parser")

    # Extraer el nombre de la zapatilla y agregarlo al diccionario.
    diccionario_zapatillas["zapatilla"].append(sopa_tienda.find("h1", {"class": "h1"}).text)

    # Extraer el precio rebajado y agregarlo al diccionario.
    diccionario_zapatillas["precio_rebajado"].append(sopa_tienda.find("span", {"class": "price"}).text)

    # Extraer el precio original y agregarlo al diccionario.
    diccionario_zapatillas["precio_original"].append(sopa_tienda.find("span", {"class": "regular-price"}).text)


# convertir el diccionario a DataFrame
df_zapatillas = pd.DataFrame(diccionario_zapatillas)
df_zapatillas

,zapatilla,precio_rebajado,precio_original
0,Zapatillas fútbol Munich Prisma 14 rojo hombre,"56,00 €","80,00 €"
1,Zapatillas futbol Ultra Play TT amarillo junior,"27,97 €","39,95 €"
2,Zapatillas fútbol Kelme Elite negro lima hombre,"44,80 €","64,00 €"
3,Zapatillas fútbol Kelme Precision blanco azul ...,"37,80 €","54,00 €"
4,Zapatillas fútbol Kelme Copa blanco azul hombre,"32,20 €","46,00 €"
5,Zapatillas fútbol Kelme Goleiro azul rojo hombre,"30,80 €","44,00 €"
6,Zapatillas fútbol Kelme Goleiro blanco azul niños,"29,40 €","42,00 €"
7,Zapatillas fútbol Kelme Precision negro naranj...,"37,80 €","54,00 €"
8,Zapatillas fútbol Joma Score 2302 blanco hombre,"36,00 €","60,00 €"
9,Zapatillas fútbol Munich Arenga 303 amarillo h...,"49,00 €","70,00 €"
